In [ ]:
trainData['Timestamp'] = pd.to_datetime(trainData['Timestamp'])
trainTrips = trainData['trips']

model = ExponentialSmoothing(trainTrips, trend='add', seasonal='add', seasonal_periods=24)

In [ ]:
#exponential smoothing model 3/9

import pandas as pd
from statsmodels.tsa.api import ExponentialSmoothing
import plotly.express as px
import numpy as np


trainData = pd.read_csv("assignment_data_train.csv")
testData = pd.read_csv("assignment_data_test.csv")

trainData['Timestamp'] = pd.to_datetime(trainData['Timestamp'])
trainTrips = trainData['trips']

model = ExponentialSmoothing(trainTrips, trend='add',
    seasonal='add', seasonal_periods=24)

modelFit = model.fit(optimized = True)

forecast1 = len(testData)
pred = modelFit.forecast(forecast1)
pred = np.array(pred)

In [ ]:
#VAR model 9/9

import pandas as pd
from statsmodels.tsa.api import VAR
import plotly.express as px
import numpy as np


trainData = pd.read_csv("assignment_data_train.csv")
testData = pd.read_csv("assignment_data_test.csv")

trainData['Timestamp'] = pd.to_datetime(trainData['Timestamp'])
trainData.set_index(pd.DatetimeIndex(trainData['Timestamp']), inplace=True)

varData = trainData[['trips','hour']].dropna()[:-50]
model = VAR(varData)

lag = model.select_order()
print(lag.summary())

modelFit = model.fit(lag.aic)

nPeriods = varData.values[-modelFit.k_ar:]
pred = modelFit.forecast(nPeriods, steps = 31*24)
pred = pred[:,0]